In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


print(plt.style.available)
%matplotlib inline
driver = webdriver.Edge()
url='https://cq.esf.fang.com/house-a058-b010742/'
driver.get(url)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [4]:

# 初始化变量
string_list = []
i = 0
Num_Pages = 20

while i < Num_Pages:
    try:
        # 等待页面加载完成

        # 找到所有的 <dl> 标签
        tables = driver.find_elements(By.TAG_NAME, 'dl')
        
        # 遍历每个 <dl> 标签，提取其 HTML 内容
        for table in tables:
            if table.get_attribute('class') == 'clearfix':
                string_list.append(table.get_attribute('outerHTML'))  # Collect raw HTML
        
        # 点击下一页
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        next_page.click()
        
        # 增加页面计数
        i += 1
    
    except NoSuchElementException:
        # 如果没有找到下一页按钮，退出循环
        print("没有找到下一页按钮，退出循环")
        break

In [5]:
string_list

['<dl class="clearfix" dataflag="bg" data-bg="{&quot;houseid&quot;:&quot;204890676&quot;,&quot;housetype&quot;:&quot;JUHE&quot;,&quot;buserid&quot;:&quot;1039324&quot;,&quot;agentid&quot;:&quot;162522909&quot;}" id="kesfqbfylb_A01_01_03"><dt class="floatl"><a ps="1_1_60" href="/chushou/3_204890676.htm" target="_blank" data_channel="2,2"><img class="loadimg" alt="(送产权车位1个)北区小高层现房,首付低7万,2梯4户17层" src="//cdnsfb.soufunimg.com/viewimage/sfb/houseverfiy/2024_12/26/M11/38/033cf25f-692c-4167-beb1-e9b83e18fb5e/224x168c.jpg?t=1" data-src="//cdnsfb.soufunimg.com/viewimage/sfb/houseverfiy/2024_12/26/M11/38/033cf25f-692c-4167-beb1-e9b83e18fb5e/224x168c.jpg?t=1" onerror="imgiserror(this,\'//cdnsfb.soufunimg.com/sfb/houseverfiy/2024_12/26/M11/38/033cf25f-692c-4167-beb1-e9b83e18fb5e.jpg\')"><p class="icon_bq"><span class="bg_red">直播看房</span></p><p class="icon_box"><span class="vr_icon"></span><span class="video_icon"></span></p></a></dt><dd><h4 class="clearfix"><a ps="1_1_60" href="/chushou/3_204890676

In [6]:


# Data processing outside the loop
all_data = []

for html in string_list:
    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # 找到<dl>标签
    dl = soup.find('dl')
    
    # 提取所需信息
    title = dl.find('h4').find('a').get_text(strip=True)
    
    # 提取房屋信息
    house_info = dl.find('p', class_='tel_shop').get_text(strip=True)
    house_info_parts = house_info.split('|')
    house_info_parts = [part.strip() for part in house_info_parts]  # 去除多余空格
    
    # 提取位置信息
    location = dl.find('p', class_='add_shop').get_text(strip=True)
    
    # 提取价格信息
    price = dl.find('span', class_='red').get_text(strip=True)
    
    # 初始化各部分的变量
    house_type = area = floor = orientation = year_built = owner = None
    
    # 根据分割后的部分数量，逐个赋值
    if len(house_info_parts) >= 1:
        house_type = house_info_parts[0]
    if len(house_info_parts) >= 2:
        area = house_info_parts[1]
    if len(house_info_parts) >= 3:
        floor = house_info_parts[2]
    if len(house_info_parts) >= 4:
        orientation = house_info_parts[3]
    if len(house_info_parts) >= 5:
        year_built = house_info_parts[4]
    if len(house_info_parts) >= 6:
        owner = house_info_parts[5]
    
    # 将提取的信息存储为字典
    data = {
        '标题': title,
        '户型': house_type,
        '面积（㎡）': area,
        '楼层': floor,
        '朝向': orientation,
        '年份': year_built,
        '户主': owner,
        '位置': location,
        '价格（元）': price
    }
    # 将字典添加到列表中
    all_data.append(data)

# 将列表转换为DataFrame
df = pd.DataFrame(all_data)
df['block'] = '空港'

In [7]:
df

,标题,户型,面积（㎡）,楼层,朝向,年份,户主,位置,价格（元）,block
0,"(送产权车位1个)北区小高层现房,首付低7万,2梯4户17层",3室2厅,81㎡,高层（共14层）,南向,2022年建,刘达现,首钢铂鹭风华空港 空港大道莲花站旁,47万,空港
1,首付7万 空港刚需必看!48万 3房送产权车位 送4万5家电,3室2厅,81㎡,中层（共14层）,南向,2024年建,祝成佳,首钢铂鹭风华空港 空港大道莲花站旁,48万,空港
2,到手40万买空港现房 送价值4万5家具家电 送产权车位,3室2厅,90㎡,中层（共17层）,东向,2023年建,祝成佳,首钢铂鹭风华空港 空港大道莲花站旁,49万,空港
3,"(国庆降21万)首付9万买3房,送产权车位 送4万5家具家电",3室2厅,81㎡,中层（共14层）,南向,2021年建,祝成佳,首钢铂鹭风华空港 空港大道莲花站旁,47万,空港
4,国企期间价!3号线轻轨口 带车位 法式 洋房 全落地窗,3室2厅,90㎡,低层（共8层）,南北向,2021年建,罗红,招商雍璟城空港 轻轨3号线莲花站,68万,空港
...,...,...,...,...,...,...,...,...,...,...
1195,原装电梯 大三室 精装 业主置换 价格可谈,4室2厅,169㎡,中层（共14层）,东向,None,None,龙凤花园空港,149万,空港
1196,"户型方正,小区安静绿化好,适合居住,出行方便",4室2厅,198.66㎡,高层（共6层）,南向,None,None,青麓雅园空港,165万,空港
1197,"空港电梯三房双卫,带花园,朝中庭,交通方便",3室1厅,118㎡,高层（共7层）,南向,None,None,瑞丰花苑空港,75.8万,空港
1198,农业园区 佳禾苑三房54.5万,3室2厅,91.87㎡,高层（共7层）,西北向,2000年建,None,空港佳禾苑空港 渝北区双凤桥路167号,54.5万,空港


In [8]:
driver.quit()

In [9]:
def chinese_to_number(s):
	if isinstance(s, str) and '万' in s:
		return float(s.replace('万', '')) * 10000
	try:
		return float(s)
	except ValueError:
		return pd.NA
        
def m_squre_to_number(s):
	if isinstance(s, str) and '㎡' in s:
		return float(s.replace('㎡', '')) 
	try:
		return float(s)
	except ValueError:
		return pd.NA

In [10]:
df['面积（㎡）'] = df['面积（㎡）'].apply(m_squre_to_number)
df['价格（元）'] = df['价格（元）'].apply(chinese_to_number)

In [11]:
df.to_parquet("konggang_housing_price.parquet", index=False)

In [12]:
df.to_csv('konggang_housing_price.csv', index=False, encoding='utf-8-sig')